# Import packages

In [4]:
import ipywidgets.widgets as widgets
from jduck import JDuck

ModuleNotFoundError: No module named 'jduck'

In [5]:
# Create slider to control jduck's speed
jduck_speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=1.0, step=0.01, orientation='vertical')

# Control buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# Display Control Widgets
middle_box = widgets.VBox([forward_button, stop_button, backward_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.HBox([jduck_speed_slider,left_button, middle_box, right_button])
display(controls_box)

# Link Control Widget to jduck

In [ ]:
def stop(change):
    jduck.stop()
    
def move_forward(change):
    jduck.move_forward()
    time.sleep(0.5)

def move_backward(change):
    jduck.move_backward()
    time.sleep(0.5)

def turn_left(change):
    jduck.turn_left()
    time.sleep(0.1)
    jduck.stop()

def turn_right(change):
    jduck.turn_right()
    time.sleep(0.1)
    jduck.stop()
# link buttons to actions
left_link = traitlets.dlink((jduck_speed_slider, 'value'), (jduck.left_motor, 'value'))
right_link = traitlets.dlink((jduck_speed_slider, 'value'), (jduck.right_motor, 'value'))
stop_button.on_click(stop)
forward_button.on_click(move_forward)
backward_button.on_click(move_backward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)